In [1]:
import json
import requests
import pandas as pd
from tqdm.notebook import tqdm
import pickle


SUBSCRIPTION_KEY = '886c69c66f8f4f2b9159a2877cb4dca4'

# Request headers
headers = {
    'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY,
}

amc_url = 'https://api.sec.or.th/FundFactsheet/fund/amc'
# 08. ประเภทกองทุนตามนโยบายกองทุน
project_policy_url = 'https://api.sec.or.th/FundFactsheet/fund/{}/policy'

In [2]:
res = requests.get(amc_url, headers=headers)
amc_list = res.json()

amc_df_dict = {
    'unique_id': [item['unique_id'] for item in amc_list],
    'name_th': [item['name_th'] for item in amc_list],
    'name_en': [item['name_en'] for item in amc_list],
    'last_upd_date': [item['last_upd_date'] for item in amc_list],
}
amc_df = pd.DataFrame(amc_df_dict)
amc_df = amc_df.set_index('unique_id')
amc_df.head()

,name_th,name_en,last_upd_date
unique_id,,,
C0000000021,บริษัท หลักทรัพย์จัดการกองทุนกสิกรไทย จำกัด,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,2020-04-08T05:04:14
C0000000023,บริษัท หลักทรัพย์จัดการกองทุนเอ็มเอฟซี จำกัด (...,MFC ASSET MANAGEMENT PUBLIC COMPANY LIMITED,2020-04-08T05:04:14
C0000000124,บริษัท หลักทรัพย์จัดการกองทุนเมอร์ชั่น พาร์ทเน...,MERCHANT PARTNERS ASSET MANAGEMENT LIMITED,2020-04-08T05:04:14
C0000000182,บริษัท หลักทรัพย์จัดการกองทุนทหารไทย จำกัด,TMB ASSET MANAGEMENT COMPANY LIMITED,2020-04-08T05:04:14
C0000000239,บริษัท หลักทรัพย์จัดการกองทุนไทยพาณิชย์ จำกัด,SCB ASSET MANAGEMENT COMPANY LIMITED,2020-04-08T05:04:14


In [3]:
amc_df.loc['C0000000021']

name_th          บริษัท หลักทรัพย์จัดการกองทุนกสิกรไทย จำกัด
name_en            KASIKORN ASSET MANAGEMENT COMPANY LIMITED
last_upd_date                            2020-04-08T05:04:14
Name: C0000000021, dtype: object

In [ ]:
# Skippable on second run
amc_project_dict = {}

for index, row in tqdm(amc_df.iterrows()):
    unique_id = index

    res = requests.get(amc_url + '/' + unique_id, headers=headers)
    project_list = res.json()

    amc_project_dict[unique_id] = project_list

with open('pickle/amc_project_dict.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(amc_project_dict))

In [4]:
# Load amc_project_dict
with open('pickle/amc_project_dict.pickle', 'rb') as pickle_file:
    amc_project_dict = pickle.load(pickle_file)

In [ ]:
# Skippable on second run
project_policy_dict = {}

for amc_id, project_list in tqdm(amc_project_dict.items()):
    for project in tqdm(project_list):
        project_id = project['proj_id']

        res = requests.get(project_policy_url.format(
            project_id), headers=headers)
        project_policy = res.json()

        project_policy_dict[project_id] = project_policy

with open('pickle/project_policy_dict.pickle', 'wb') as pickle_file:
    pickle_file.write(pickle.dumps(project_policy_dict))

In [5]:
# Load project_policy_dict
with open('pickle/project_policy_dict.pickle', 'rb') as pickle_file:
    project_policy_dict = pickle.load(pickle_file)

project_policy_dict['M0000_2553']

{'last_upd_date': '2020-04-01T05:03:34',
 'policy_desc': 'ตราสารแห่งหนี้',
 'management_style': '-',
 'investment_policy_desc': 'LQ=='}

In [6]:
import os.path

project_performance_dict_pickle_path = 'pickle/project_performance_dict.pickle'
project_investment_dict_pickle_path = 'pickle/project_investment_dict.pickle'
project_benchmark_dict_pickle_path = 'pickle/project_benchmark_dict.pickle'


def download_project_attribute(id_df, api_url, pickle_path):
    """
    id_df is a DataFrame wtih "amc_id" and "project_id" columns
    """

    if os.path.isfile(pickle_path):
        with open(pickle_path, 'rb') as pickle_file:
            project_attr_dict = pickle.load(pickle_file)
    else:
        project_attr_dict = {}

    for index, row in tqdm(id_df.iterrows(), total=id_df.shape[0]):
        amc_id = row['amc_id']
        project_id = row['project_id']

        if project_id in project_attr_dict.keys():
            continue

        res = requests.get(api_url.format(
            proj_id=project_id), headers=headers)
        if res.status_code == 200:
            project_attr = res.json()
        else:
            continue

        project_attr_dict[project_id] = project_attr
                
    with open(pickle_path, 'wb') as pickle_file:
        pickle_file.write(pickle.dumps(project_attr_dict))
                    
def load_pickle(pickle_path):
    with open(pickle_path, 'rb') as pickle_file:
        return pickle.load(pickle_file)

def load_project_performance():
    return load_pickle(project_performance_dict_pickle_path)
    
def load_project_investment():
    return load_pickle(project_investment_dict_pickle_path)

def load_project_benchmark():
    return load_pickle(project_benchmark_dict_pickle_path)

In [38]:
amc_project_df_dict = {
    'amc_id': [],
    'amc_name': [],
    'project_id': [],
    'project_name': [],
    'fund_status':[],
    'management_style': [],
    'policy_desc': []
}

for amc_id, project_list in tqdm(amc_project_dict.items()):
    amc = amc_df.loc[amc_id]
    amc_name = amc['name_en']
    for project in project_list:
        project_id = project['proj_id']
        project_name = project['proj_name_en']
        fund_status=project['fund_status']

        amc_project_df_dict.get('amc_id').append(amc_id)
        amc_project_df_dict.get('amc_name').append(amc_name)
        amc_project_df_dict.get('project_id').append(project_id)
        amc_project_df_dict.get('project_name').append(project_name)
        amc_project_df_dict.get('fund_status').append(fund_status)

        management_style = project_policy_dict[project_id]['management_style']

        amc_project_df_dict.get('management_style').append(management_style)
        
        policy_desc = project_policy_dict[project_id]['policy_desc']

        amc_project_df_dict.get('policy_desc').append(policy_desc)

amc_project_df = pd.DataFrame(amc_project_df_dict)
amc_project_df.head()

,amc_id,amc_name,project_id,project_name,fund_status,management_style,policy_desc
0,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0000_2553,K Fixed Income 3 Months B Fund,LI,-,ตราสารแห่งหนี้
1,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0001_2558,K Enhanced Foreign Fixed Income 1 Year AD Fund,LI,-,ตราสารแห่งหนี้
2,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2558,K Enhanced Fixed Income 3 Months K Fund,LI,-,ตราสารแห่งหนี้
3,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2563,K Foreign Fixed Income 6 Months HX Fund,EX,BH,ตราสารหนี้
4,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0004_2555,K Fixed Income 6 Months AJ Fund,LI,-,ตราสารแห่งหนี้


In [39]:
amc_project_df.to_csv('csv/amc_project.csv', index=False)

In [40]:
amc_project_df = pd.read_csv('csv/amc_project.csv')
amc_project_df.head()

,amc_id,amc_name,project_id,project_name,fund_status,management_style,policy_desc
0,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0000_2553,K Fixed Income 3 Months B Fund,LI,-,ตราสารแห่งหนี้
1,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0001_2558,K Enhanced Foreign Fixed Income 1 Year AD Fund,LI,-,ตราสารแห่งหนี้
2,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2558,K Enhanced Fixed Income 3 Months K Fund,LI,-,ตราสารแห่งหนี้
3,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0003_2563,K Foreign Fixed Income 6 Months HX Fund,EX,BH,ตราสารหนี้
4,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0004_2555,K Fixed Income 6 Months AJ Fund,LI,-,ตราสารแห่งหนี้


In [33]:
amc_project_df[amc_project_df['management_style'] == 'AM']

,amc_id,amc_name,project_id,project_name,fund_status,management_style,policy_desc
12,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0008_2537,THE RUANG KHAO 4 FUND,RG,AM,ตราสารทุน
17,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0009_2543,K Flexible Equity Fund,RG,AM,ผสม
23,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0014_2536,K Select Equity Fund,RG,AM,ตราสารแห่งทุน
47,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0023_2538,K Valued Stock Fund,RG,AM,ตราสารทุน
54,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0027_2535,K Equity Fund,RG,AM,ตราสารทุน
...,...,...,...,...,...,...,...
11055,C0000028866,WE ASSET MANAGEMENT COMPANY LIMITED,M0246_2562,WE CHINA 6M FUND,RG,AM,ผสม
11057,C0000028866,WE ASSET MANAGEMENT COMPANY LIMITED,M0310_2562,WE INDIA 9M FUND,RG,AM,ผสม
11058,C0000028866,WE ASSET MANAGEMENT COMPANY LIMITED,M0318_2562,WE GLOBAL TECHNOLOGY 8M FUND,LI,AM,ตราสารทุน
11059,C0000028866,WE ASSET MANAGEMENT COMPANY LIMITED,M0354_2562,WE GLOBAL ALLOCATION 5M FUND,RG,AM,ผสม


In [46]:
am_filter = (amc_project_df.management_style == 'AM') & (
    amc_project_df.fund_status == 'RG') & (amc_project_df.policy_desc == 'ตราสารทุน')
pm_filter = (amc_project_df.management_style == 'PM') & (
    amc_project_df.fund_status == 'RG') & (amc_project_df.policy_desc == 'ตราสารทุน')
pm_filter = (amc_project_df.management_style == 'PM') & (
    amc_project_df.fund_status == 'RG')

amc_project_df.where(pm_filter).dropna().reset_index()

,index,amc_id,amc_name,project_id,project_name,fund_status,management_style,policy_desc
0,311,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0140_2558,K Energy Sector Index Fund,RG,PM,ตราสารแห่งทุน
1,319,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0145_2549,The ABF Thailand Bond Index Fund,RG,PM,ตราสารแห่งหนี้
2,463,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0209_2548,K SET 50 Index Fund,RG,PM,ตราสารแห่งทุน
3,566,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0250_2558,K ICT Sector Index Fund,RG,PM,ตราสารแห่งทุน
4,847,C0000000021,KASIKORN ASSET MANAGEMENT COMPANY LIMITED,M0373_2558,K Banking Sector Index Fund,RG,PM,ตราสารแห่งทุน
...,...,...,...,...,...,...,...,...
128,10921,C0000006646,LAND AND HOUSES FUND MANAGEMENT COMPANY LIMITED,M0066_2562,LH THAI ESG FUND,RG,PM,ตราสารทุน
129,11022,C0000021747,BANGKOK CAPITAL ASSET MANAGEMENT COMPANY LIMITED,M0050_2558,BCAP MSCI THAILAND ETF,RG,PM,ตราสารแห่งทุน
130,11030,C0000021747,BANGKOK CAPITAL ASSET MANAGEMENT COMPANY LIMITED,M0259_2560,BCAP SET100 ETF,RG,PM,ตราสารทุน
131,11031,C0000021747,BANGKOK CAPITAL ASSET MANAGEMENT COMPANY LIMITED,M0356_2561,BCAP MSCI Thailand LTF,RG,PM,ตราสารทุน


In [41]:
# Skippable on second run

project_data_configs = [
    # (name, url, pickle_path)
    # 06. มูลค่าหน่วยลงทุน/จำนวนหน่วยลงทุนในการสั่งซื้อขาย/คงเหลือ
    ('investment', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/investment', 'pickle/project_investment_dict.pickle'),
    # 08. ประเภทกองทุนตามนโยบายกองทุน
    ('policy', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/policy', 'pickle/project_policy_dict.pickle'),
    # 18. ดัชนีชี้วัดกองทุน
    ('benchmark', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/benchmark', 'pickle/project_benchmark_dict.pickle'),
    # 19. ประเภทกองทุนรวมเพื่อใช้เปรียบเทียบผลการดำเนินงาน ณ จุดขาย
    ('fund_compare', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/fund_compare', 'pickle/project_fund_compare_dict.pickle'),
    # 22. ผลการดำเนินงานย้อนหลังของกองทุน
    ('performace', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/performance', 'pickle/project_performance_dict.pickle'),
    # 23. ผลขาดทุนสูงสุดในช่วง 5 ปีและความผันผวนของผลการดำเนินงานย้อนหลัง 5 ปี
    ('5YearLost', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/5YearLost', 'pickle/project_5YearLost_dict.pickle'),
    # 25. ค่าธรรมเนียมกองทุน
    ('fee', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/fee', 'pickle/project_fee_dict.pickle'),
]

project_data_configs_pm = [
    # 18. ดัชนีชี้วัดกองทุน
    ('benchmark', 'https://api.sec.or.th/FundFactsheet/fund/{proj_id}/benchmark', 'pickle/project_benchmark_dict.pickle'),
]

# Download all project's attributes based on configs
for config in project_data_configs:
    name, url, pickle_path = config
    print("Downloading {}...".format(name))
    download_project_attribute(amc_project_df.where(am_filter).dropna().reset_index(), url, pickle_path)
    
for config in project_data_configs_pm:
    name, url, pickle_path = config
    print("Downloading {}...".format(name))
    download_project_attribute(amc_project_df.where(pm_filter).dropna().reset_index(), url, pickle_path)